<a href="https://colab.research.google.com/github/julia-freitas/TCC-2023/blob/main/01_tratamento_com_base_raw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Etapas do projeto

* Data Collection
    * leitura de arquivo

* Data Preparation

    * EDA
        * Estudo de correlação das variáveis
        * histograma das doenças em comorbidade
        * histograma de agrupamento de idades
        * histograma idades x genero

    * Data preprocessing (Transformação de dados)
        * Dados nulos (dados faltantes)
        * String em coluna numérica (como tratar?)
        * Normalização dos Dados  - Normalization vs Standardization
        * Balanceamento das Classes (SMOTE e variantes)
        * Engenharia de Variáveis (Feature Engineering)

    * Data Splitting


* Treinamento dos modelos
    * Escolha dos algoritmos
    * Overfit e regularização
    * Tuning de Hiperparametros

* Avaliação dos modelos
    
    * Avaliação dos algoritmos
        * Matriz de Confusão (porcentagens de TP, FP, TN, FN)
        * Classification
            * acuracia = (TP+TN)/(TP+ TN + FP + FN)
            * precisão = TP / (TP + FP)
            * recall = TP / (TP + FN)
            * f1-score = (2 * Precision X recall) / (precision + recall)
        
    * Feature Importance
    * Training/inference cost



#### Import das bases

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style('darkgrid')
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [15]:
df = pd.read_excel('/content/drive/MyDrive/TCC/base/base-raw-UKDA-8502-stata/csv/Cópia de hcap_2018_eul_respondent_archive.xlsx', sheet_name='base-apos-limpeza')

In [16]:
df.columns

Index(['idauniq', 'indager', 'sex', 'hehelf', 'hediabp', 'hediaan', 'hediami',
       'hediahf', 'hediadi', 'hediast', 'hedibpd', 'hedibps', 'hedibad',
       'hedibde', 'hedibca', 'hediblu', 'hedi96', 'heeye', 'hehear', 'hessme',
       'cfmetm', 'headldr', 'headlwa', 'headlba', 'headlea', 'headlbe',
       'headlwc', 'headlma', 'headlda', 'headlpr', 'headlsh', 'headlph',
       'headlsp', 'headlme', 'headlho', 'headlmo', 'headl96', 'CESD_score',
       'adl_score', 'iadl_score', 'MMSE_score'],
      dtype='object')

#### Renomeando as colunas

In [17]:
df.rename(columns={
"hehelf" : "saude_geral" ,
"hediabp" : "hipertensao" ,
"hediaan" : "angina" ,
"hediami" : "ataque_cardiaco" ,
"hediahf" : "insuf_cardiaca" ,
"hediadi" : "diabetes" ,
"hediast" : "AVC" ,
"hedibpd" : "parkinson" ,
"hedibps" : "emotional" ,
"hedibad" : "alzheimer" ,
"hedibde" : "dementia" ,
"hedibca" : "cancer" ,
"hediblu" : "doenca_respiratoria" ,
"hedi96" : "nenhuma_doenca" ,
"heeye" : "visao" ,
"hehear" : "audicao" ,
"hessme" : "olfato" ,
"cfmetm" : "memoria" ,
"headldr" : "adl_vestir" ,
"headlwa" : "adl_caminhar" ,
"headlba" : "adl_banho" ,
"headlea" : "adl_comer" ,
"headlbe" : "adl_sair_cama" ,
"headlwc" : "adl_banheiro" ,
"headlma" : "iadl_mapa" ,
"headlda" : "iadl_perigo" ,
"headlpr" : "iadl_prep_comida" ,
"headlsh" : "iadl_mercado" ,
"headlph" : "iadl_telefonemas" ,
"headlsp" : "iadl_fala" ,
"headlme" : "iadl_medicacao" ,
"headlho" : "iadl_serv_domestico" ,
"headlmo" : "iadl_financas" ,
"headl96" : "iadl_nenhuma"
},

inplace=True
)

### Exploracao dos dados


In [18]:
df.head()

,idauniq,indager,sex,saude_geral,hipertensao,angina,ataque_cardiaco,insuf_cardiaca,diabetes,AVC,...,iadl_telefonemas,iadl_fala,iadl_medicacao,iadl_serv_domestico,iadl_financas,iadl_nenhuma,CESD_score,adl_score,iadl_score,MMSE_score
0,106269,83,Male,3,0,0,0,0,0,0,...,0,0,0,0,0,0,4,1,1,24
1,106891,82,Female,3,0,0,0,0,0,0,...,0,0,0,1,0,0,2,2,2,22
2,108662,75,Female,5,0,0,0,0,0,0,...,0,0,1,0,1,0,11,4,2,23
3,105965,74,Female,3,0,0,0,0,0,0,...,0,0,0,0,0,1,4,0,0,25
4,105070,78,Male,1,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,30


In [19]:
df['saude_geral'] = df['saude_geral'].astype(object)
#df['hipertensao'] = df['hipertensao'].astype(object)
#df['angina'] = df['angina'].astype(object)
#df['ataque_cardiaco'] = df['ataque_cardiaco'].astype(object)
#df['insuf_cardiaca'] = df['insuf_cardiaca'].astype(object)
#df['diabetes'] = df['diabetes'].astype(object)
#df['AVC'] = df['AVC'].astype(object)
#df['parkinson'] = df['parkinson'].astype(object)
#df['emotional'] = df['emotional'].astype(object)
#df['alzheimer'] = df['alzheimer'].astype(object)
#df['dementia'] = df['dementia'].astype(object)
#df['cancer'] = df['cancer'].astype(object)
#df['doenca_respiratoria'] = df['doenca_respiratoria'].astype(object)
#df['nenhuma_doenca'] = df['nenhuma_doenca'].astype(object)
df['visao'] = df['visao'].astype(object)
df['audicao'] = df['audicao'].astype(object)
df['olfato'] = df['olfato'].astype(object)
df['memoria'] = df['memoria'].astype(object)
df['adl_vestir'] = df['adl_vestir'].astype(object)
df['adl_caminhar'] = df['adl_caminhar'].astype(object)
df['adl_banho'] = df['adl_banho'].astype(object)
df['adl_comer'] = df['adl_comer'].astype(object)
df['adl_sair_cama'] = df['adl_sair_cama'].astype(object)
df['adl_banheiro'] = df['adl_banheiro'].astype(object)
df['iadl_mapa'] = df['iadl_mapa'].astype(object)
df['iadl_perigo'] = df['iadl_perigo'].astype(object)
df['iadl_prep_comida'] = df['iadl_prep_comida'].astype(object)
df['iadl_mercado'] = df['iadl_mercado'].astype(object)
df['iadl_telefonemas'] = df['iadl_telefonemas'].astype(object)
df['iadl_fala'] = df['iadl_fala'].astype(object)
df['iadl_medicacao'] = df['iadl_medicacao'].astype(object)
df['iadl_serv_domestico'] = df['iadl_serv_domestico'].astype(object)
df['iadl_financas'] = df['iadl_financas'].astype(object)
df['iadl_nenhuma'] = df['iadl_nenhuma'].astype(object)

### Limpeza de Dados

Quantos atributos e quantas entradas o nosso conjunto de dados possui? Quais os tipos de dados?

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1273 entries, 0 to 1272
Data columns (total 41 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   idauniq              1273 non-null   int64 
 1   indager              1273 non-null   int64 
 2   sex                  1273 non-null   object
 3   saude_geral          1273 non-null   object
 4   hipertensao          1273 non-null   int64 
 5   angina               1273 non-null   int64 
 6   ataque_cardiaco      1273 non-null   int64 
 7   insuf_cardiaca       1273 non-null   int64 
 8   diabetes             1273 non-null   int64 
 9   AVC                  1273 non-null   int64 
 10  parkinson            1273 non-null   int64 
 11  emotional            1273 non-null   int64 
 12  alzheimer            1273 non-null   int64 
 13  dementia             1273 non-null   int64 
 14  cancer               1273 non-null   int64 
 15  doenca_respiratoria  1273 non-null   int64 
 16  nenhum

In [21]:
df.describe()

,idauniq,indager,hipertensao,angina,ataque_cardiaco,insuf_cardiaca,diabetes,AVC,parkinson,emotional,alzheimer,dementia,cancer,doenca_respiratoria,nenhuma_doenca,CESD_score,adl_score,iadl_score,MMSE_score
count,1273.00,1273.00,1273.00,1273.00,1273.00,1273.00,1273.00,1273.00,1273.00,1273.00,1273.00,1273.00,1273.00,1273.00,1273.00,1273.00,1273.00,1273.00,1273.00
mean,122373.13,566.81,0.15,0.02,0.02,0.01,0.05,0.02,0.01,0.01,0.01,0.02,0.05,0.04,0.69,1.99,0.40,0.67,25.95
std,20395.93,2407.13,0.36,0.15,0.15,0.09,0.23,0.15,0.10,0.11,0.12,0.16,0.22,0.21,0.46,2.29,1.05,1.46,3.80
min,103742.00,65.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,0.00,0.00,2.00
25%,107701.00,70.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,24.00
50%,116910.00,75.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00,27.00
75%,120581.00,81.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,3.00,0.00,1.00,29.00
max,162571.00,12345.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,11.00,6.00,9.00,30.00


### Substituindo variavel categorica, por numerica

In [22]:
df['sex'] = df['sex'].apply(lambda x: 1 if x == 'Male' else (2 if x == 'Female' else x))

In [23]:
df_sem_nulos = df[df['hipertensao'] >= 0]
df_sem_nulos = df[df['diabetes'] >= 0]
df_sem_nulos = df[df['alzheimer'] >= 0]
df_sem_nulos = df[df['dementia'] >= 0]
df_sem_nulos = df[df['CESD_score'] >= 0]

### Salvando a base de dados tratada em um arquivo csv

In [24]:
#salvando os dados do dataframe em um arquivo csv
df.to_csv('/content/drive/MyDrive/TCC/base/base-raw-UKDA-8502-stata/csv/base-apos-tratamento.csv', index=False)